In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import json

In [ ]:
import nltk
import nltk
nltk.download('punkt')  

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
path ='/content/gdrive/MyDrive/thesis python/snli_1.0/'

# Preparing data for fine-tuning

In [ ]:
labels =['entailment', 'contradiction', 'neutral']
label_dict = {'contradiction': 0, 'entailment': 1, 'neutral': 2}

In [ ]:
# this is the original train data and 50k*6 new examples
with open( path + '50k_augmented_dataset.jsonl') as f:
  aug_list =[]
  for line in f:
    line =json.loads(line)
    if line['gold_label'] in labels:
      aug_list.append(line) 
print(len(aug_list))

849328


In [ ]:
with open(path +"aug.jsonl", "w+") as outfile: 
  pass

In [ ]:
# save in new file where label is identified by number
new_f= open(path + "aug.jsonl", 'a+')
for line in aug_list:
  json_line=line
  num =label_dict[json_line['gold_label']]
  new_line = {'hypothesis' : json_line.get('sentence2'), 'idx' : json_line['pairID'], 'label' : num, 'premise' : json_line['sentence1']}
  json_line = json.dumps(new_line)
  new_f.write(json_line)
  new_f.write('\n')

In [ ]:
#check if amount of line in the aug.jsonl file just created
with open(path +'aug.jsonl') as f:
  aug_file =[]
  for line in f:
    line =json.loads(line)
    aug_file.append(line)
print(len(aug_file))

849263


In [ ]:
new_f= open(path +"snli_1.0_test.jsonl",'r')
test_list=[]
for line in new_f:
  line =json.loads(line)
  if line['gold_label'] in labels:
    test_list.append(line)
print(len(test_list))

9824


In [ ]:
with open(path +"test_set.jsonl", "w+") as outfile: 
  pass

In [ ]:
new_f= open(path +"test_set.jsonl", 'a+')
for line in test_list:
  json_line=line
  num =label_dict[json_line['gold_label']]
  new_line = {'hypothesis' : json_line.get('sentence2'), 'idx' : json_line['pairID'], 'label' : num, 'premise' : json_line['sentence1']}
  json_line = json.dumps(new_line)
  new_f.write(json_line)
  new_f.write('\n')

In [ ]:
#check if amount of line in the test_set.jsonl file just created
with open(path +'test_set.jsonl') as f:
  test_file =[]
  for line in f:
    line =json.loads(line)
    test_file.append(line)
print(len(test_file))

9694


In [ ]:
new_f= open(path +"snli_1.0_dev.jsonl",'r')
dev_list=[]
for line in new_f:
  line =json.loads(line)
  if line['gold_label'] in labels:
    dev_list.append(line)
print(len(dev_list))

9842


In [ ]:
with open(path +"dev_set.jsonl", "w+") as outfile: 
  pass

In [ ]:
new_f= open(path +"dev_set.jsonl", 'a+')
for line in dev_list:
  json_line=line
  num =label_dict[json_line['gold_label']]
  new_line = {'hypothesis' : json_line.get('sentence2'), 'idx' : json_line['pairID'], 'label' : num, 'premise' : json_line['sentence1']}
  json_line = json.dumps(new_list)
  new_f.write(json_line)
  new_f.write('\n')

In [ ]:
#check if amount of line in the dev_set.jsonl file just created
with open(path +'dev_set.jsonl') as f:
  dev_file =[]
  for line in f:
    line =json.loads(line)
    dev_file.append(line)
print(len(dev_file))

9684


# Fine-tuning DistilBERT on aug SNLI

In [ ]:
import torch
from os import path as op
import os

In [ ]:
# transformers complained about newset version 0.0.13 so installing the older version
! pip install huggingface-hub==0.0.12 

In [ ]:
! pip install datasets transformers

     |████████████████████████████████| 245kB 17.9MB/s 
     |████████████████████████████████| 2.5MB 37.4MB/s 
     |████████████████████████████████| 245kB 53.3MB/s 
     |████████████████████████████████| 122kB 59.0MB/s 
     |████████████████████████████████| 3.3MB 46.5MB/s 
     |████████████████████████████████| 901kB 43.5MB/s 


In [ ]:
import datasets
from datasets import load_dataset, load_metric
import numpy as np
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

In [ ]:
# META Variables
MODEL_DIR = 'model_checkpoints' 
MODEL_CHECKPOINT = "distilbert-base-uncased"
BATCH_SIZE = 16


In [ ]:
dev = load_dataset('json', data_files= path + "dev_set.jsonl")

Using custom data configuration default-e497f20cea47b59d


Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-e497f20cea47b59d/0.0.0/83d5b3a2f62630efc6b5315f00f20209b4ad91a00ac586597caee3a4da0bef02. Subsequent calls will reuse this data.


In [ ]:
ts = load_dataset('json', data_files= path + "test_set.jsonl")

Using custom data configuration default-dcb2d6a3854e3c4e


Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-dcb2d6a3854e3c4e/0.0.0/83d5b3a2f62630efc6b5315f00f20209b4ad91a00ac586597caee3a4da0bef02. Subsequent calls will reuse this data.


In [ ]:
dataset = load_dataset('json', data_files= path + "aug.jsonl")

In [ ]:
metric = load_metric('glue', "mnli")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT, use_fast=True)

In [ ]:
# https://huggingface.co/transformers/preprocessing.html
def preprocess_function(d):
    return tokenizer(d['premise'], d['hypothesis'], truncation=True)

In [ ]:
encoded_dataset = dataset.map(preprocess_function, batched=True, load_from_cache_file=True)

In [ ]:
encoded_dev = dev.map(preprocess_function, batched=True, load_from_cache_file=True)

In [ ]:
encoded_ts = ts.map(preprocess_function, batched=True, load_from_cache_file=True)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL_CHECKPOINT, num_labels=3)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'pre_classi

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
args = TrainingArguments(
    MODEL_DIR, # to save models
    evaluation_strategy = "steps", # 1 epoch for training takes too long for colab
    eval_steps = 5000, # evaluate and save after training on every next 5000x16 examples
    learning_rate=2e-5,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=1,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dev['train'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

# Evaluation (no fine-tuning) aug SNLI

In [ ]:
# if you want to load a model from a checkpoint for evaluation
# ft_model = AutoModelForSequenceClassification.from_pretrained(op.join(MODEL_DIR, 'checkpoint-5000'))

In [ ]:
# evaluation of a particular model
trainer_eval = Trainer(
    model, # model that you want to evaluate
    args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_ts['train'], # you want to evaluate on test
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer_eval.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: premise, hypothesis, idx.
***** Running Evaluation *****
  Num examples = 9694
  Batch size = 16


{'eval_accuracy': 0.88405199092222,
 'eval_loss': 0.3163028061389923,
 'eval_runtime': 14.2202,
 'eval_samples_per_second': 681.707,
 'eval_steps_per_second': 42.615}

# Fine-tuning DistilBERT on SNLI

In [ ]:
dataset = load_dataset('json', data_files= path + "train_set.jsonl")

Using custom data configuration default-463e97adf3859c5c


Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-463e97adf3859c5c/0.0.0/83d5b3a2f62630efc6b5315f00f20209b4ad91a00ac586597caee3a4da0bef02. Subsequent calls will reuse this data.


In [ ]:
encoded_dataset = dataset.map(preprocess_function, batched=True, load_from_cache_file=True)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL_CHECKPOINT, num_labels=3)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'pre_classi

In [ ]:
args = TrainingArguments(
    MODEL_DIR,
    evaluation_strategy = "steps",
    eval_steps = 5000,
    learning_rate=2e-5,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=1,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dev['train'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: premise, hypothesis, idx.
***** Running training *****
  Num examples = 549034
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 34315


Step,Training Loss,Validation Loss,Accuracy
5000,0.354200,0.341091,0.876291
10000,0.351300,0.327392,0.882280
15000,0.347900,0.325779,0.881660


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: premise, hypothesis, idx.
***** Running Evaluation *****
  Num examples = 9684
  Batch size = 16
Saving model checkpoint to model_checkpoints/checkpoint-5000
Configuration saved in model_checkpoints/checkpoint-5000/config.json
Model weights saved in model_checkpoints/checkpoint-5000/pytorch_model.bin
tokenizer config file saved in model_checkpoints/checkpoint-5000/tokenizer_config.json
Special tokens file saved in model_checkpoints/checkpoint-5000/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: premise, hypothesis, idx.
***** Running Evaluation *****
  Num examples = 9684
  Batch size = 16
Saving model checkpoint to model_checkpoints/checkpoint-10000
Configuration saved in model_checkpoints/checkpoint-10000/c

# Evaluation (no fine-tuning) SNLI

In [ ]:
# if you want to load a model from a checkpoint for evaluation
# ft_model = AutoModelForSequenceClassification.from_pretrained(op.join(MODEL_DIR, 'checkpoint-5000'))

In [ ]:
# evaluation of a particular model
trainer_eval = Trainer(
    model, 
    args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_ts['train'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer_eval.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: premise, hypothesis, idx.
***** Running Evaluation *****
  Num examples = 9694
  Batch size = 16


{'eval_accuracy': 0.88405199092222,
 'eval_loss': 0.3163028061389923,
 'eval_runtime': 14.2202,
 'eval_samples_per_second': 681.707,
 'eval_steps_per_second': 42.615}

# Continue training (if colab crashed)

In [ ]:
# if colab timeouts after one evaluation (i.e., training on 5000x16),
# you will still have a model in $MODEL_DIR/checkpoint-5000
# you can load that model and continue fine-tuning on the remaining problems
# note that the first 5000x16 problems will be skipped

# trainer.train(op.join(MODEL_DIR, 'checkpoint-5000'))